In [1]:
os.getcwd() 

'/Users/biancaziliotto/Desktop/GitRepo/MLx2'

In [2]:
os.chdir('src')

In [3]:
from MLP import MLP 

In [4]:
mlp = MLP([4, 5, 9, 4, 7], 3, 2, "Sigm")


4
5
9
4
7
2


In [5]:
mlp.layers

In [6]:
import numpy as np
X = np.zeros((4,3))
y_true  = X[:,1:]
X[0][0] =1
print(X)
print(y_true)

[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [7]:
mlp.fit(X, y_true, batch_size=2, initialization_str = "scaled", scale = 0.1, error_function_str= "MSE", optimizer_str = "gradient_descent", regularization_function_str= "L1")

MSE = 0.028597683789413526
[[ 0.06328951 -0.23062916]
 [ 0.06328953 -0.23062914]]
[[0. 0. 0. 0.]]


ValueError: operands could not be broadcast together with shapes (2,2) (1,4) 

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])